<a href="https://colab.research.google.com/github/lonemagnolia/lonemagnolia/blob/main/2024_06_26_Biopython_EAPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I installed Biopython and Seaborn  using the Gemini helper.

My goal for the day is to just get started somewhere. I found the the rRNA sequences on the NCBI datasets.

https://www.ncbi.nlm.nih.gov/datasets/gene/GCF_000006945.2/?gene_type=rRNA

It's a reference sequence for Salmonella enterica subsp. enterica serovar Typhimurium str. LT2 ASM694v2 (GCF_000006945.2)
Annotation Name: Annotation submitted by NCBI RefSeq (September 7, 2022)

I picked that because I have the most familiarity with Salmonella and rRNA is relatively small and well documented.

I'm following BioPython's Cookbook for now.


In [4]:

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import Bio

This is following Chapter 2.2 in the Biopython Docs.

In [7]:
from Bio.Seq import Seq
my_seq = Seq("AGTACACTGGT")
my_seq
Seq('AGTACACTGGT')
print(my_seq)


AGTACACTGGT


In [23]:
my_seq = Seq("AGTACACTGGT")
my_seq.complement()
my_seq.reverse_complement()
print("this is the complement " + my_seq.complement())

print("This is the reverse complement " + my_seq.reverse_complement())



this is the complement TCATGTGACCA
This is the reverse complement ACCAGTGTACT


Now we are getting more interesting. The first thing is a way to count the fraction of G and C in the string. This is useful for a multitude of reasons, but experimentally, it's the fact that you may need a higher temperature/ more time using PCR to dissolve the hydrogen bonds between G and C.

In [24]:
from Bio.Seq import Seq
from Bio.SeqUtils import gc_fraction
my_seq = Seq("GATCGATGGGCCTATATAGGATCGAAAATCGC")
gc_fraction(my_seq)

0.46875

Practicing slicing a sequence:



In [30]:

my_seq = Seq("GATCGATGGGCCTATATAGGATCGAAAATCGC")
print(my_seq[4:12])
print(my_seq[0:3])
print(my_seq[0::3])
print(my_seq[1::3])
print(my_seq[2::3])



GATGGGCC
GAT
GCTGTAGTAAG
AGGCATGCATC
TAGCTAAGAC


Continuing my read through the doc, I noticed something I have never seen before the 5'-3' strand is referred to as the "Watson Strand" (used as the template strand during transcription) and the 3'-5' strand is called the "Crick Strand".

So this is all to say

5'-3' is referred to as
*   the Watson strand
*   strand -1
*   template strand (for transcription)

3'-5' is referred to as:
*   the Crick Strand
*   strand +1
*   the coding strand

So we have a coding DNA Sequence, and when we use reverse complement we get the template sequence, but it is written **5'-3'** not 3'-5'.

Note: We can mimic biological transcription using the template dna by using reverse_complement and transcribe.



In [33]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
print(coding_dna)
template_dna = coding_dna.reverse_complement()
print(template_dna)
m_rna= coding_dna.transcribe()
print(m_rna)

ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG
CTATCGGGCACCCTTTCAGCGGCCCATTACAATGGCCAT
AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG


So we can get the reverse transcription to go from RNA to DNA using back_transcribe. SO NEAT!

In [35]:
messenger_rna = Seq("AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG")
print(messenger_rna)
messenger_dna = messenger_rna.back_transcribe()
print(messenger_dna)




AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG


Next up is translation.

So the stars represent stop codons, but can be changed to your desired symbol.  

The codon table that is used without specifying is the traditional codon table from NCBI.

BUT!!! You can specify your desired codon table from the NCBI. [Genetic Codes from NCBI](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi)


So the number associated can be used instead of the name of the table itself, but I wanted to show it here.

As a result of using the vertebrate mitochondrial table instead of the traditional, all purpose table we see that the middle stop codon is now tryptophan.


I wonder if you can add your own? In cases where the genetic code has been manipulated by manipulating the tRNA. Will explore at another time.


In [37]:
messenger_rna = Seq("AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG")
print(messenger_rna)
protein1 = messenger_rna.translate()
print(protein1)
protein_mito = messenger_rna.translate(table="Vertebrate Mitochondrial")
print(protein_mito)



AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
MAIVMGR*KGAR*
MAIVMGRWKGAR*


You can specify that you want the translation to stop at the first stop codon.
The stop codon is not included in the output.

In [41]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
print("coding DNA " + coding_dna)
protein2= coding_dna.translate()
print( "protein 2 " + protein2)
protein2_trad = coding_dna.translate(to_stop=True)
print("traditional protein 2 " + protein2_trad)
protein2_mito = coding_dna.translate(table=2, to_stop=True)
print( "Mitochondrial protein 2 " + protein2_mito)


coding DNA ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG
protein 2 MAIVMGR*KGAR*
traditional protein 2 MAIVMGR
Mitochondrial protein 2 MAIVMGRWKGAR


Complete coding sequence (CDS) = "a nucleotide sequence (e.g. mRNA – after any splicing) which is a whole number of codons (i.e. the length is a multiple of three), commences with a start codon, ends with a stop codon, and has no internal in-frame stop codons." Chapter 3.8- Biopython tutorial

Nonstandard start codons are present especially in bacteria, so in order to get the proper output we can specify that the gene is a CDS, which we did with cds=true.

In [45]:
gene = Seq(
...     "GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGGTCGCTCCCATGGCA"
...     "GCACAGGCTGCGGAAATTACGTTAGTCCCGTCAGTAAAATTACAGATAGGCGATCGTGAT"
...     "AATCGTGGCTATTACTGGGATGGAGGTCACTGGCGCGACCACGGCTGGTGGAAACAACAT"
...     "TATGAATGGCGAGGCAATCGCTGGCACCTACACGGACCGCCGCCACCGCCGCGCCACCAT"
...     "AAGAAAGCTCCTCATGATCATCACGGCGGTCATGGTCCAGGCAAACATCACCGCTAA"
... )
protein3 = gene.translate(table="Bacterial")
print (protein3)

protein3_stop = gene.translate(table="Bacterial", to_stop=True)
print (protein3_stop)

protein3_CDS = gene.translate(table="Bacterial", cds=True)
print (protein3_CDS)

VKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDHGWWKQHYEWRGNRWHLHGPPPPPRHHKKAPHDHHGGHGPGKHHR*
VKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDHGWWKQHYEWRGNRWHLHGPPPPPRHHKKAPHDHHGGHGPGKHHR
MKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDHGWWKQHYEWRGNRWHLHGPPPPPRHHKKAPHDHHGGHGPGKHHR


Next up is Finding subsequences in a sequence.

Seq are immutable (for good reason), and we can search a sequence in a couple different ways.



If the subsequence is not present then the seq.index() will return an error.

In [46]:
from Bio.Seq import Seq, MutableSeq
seq = Seq("GCCATTGTAATGGGCCGCTGAAAGGGTGCCCGA")
seq.index("ATGGGCCGC")
## Searching for the sequence in the object seq.
##It returns the index of where it starts
##Will throw a value error if the subsequence isn't found

seq.index(b"ATGGGCCGC")
## Searching for the sequence, but it's a byte now, in the object seq.
##It returns the index of where it starts
seq.index(bytearray(b"ATGGGCCGC"))
## Searching for the sequence, but it's a byte array now, in the object seq.
##It returns the index of where it starts
seq.index(Seq("ATGGGCCGC"))
## Searching for the sequence, but it's a sequence now, in the object seq.
##It returns the index of where it starts
seq.index(MutableSeq("ATGGGCCGC"))
## Searching for the sequence, but it's a mutable sequence now, in the object seq.
##It returns the index of where it starts

9

Other options are find, rfind, or rindex.

Search is used to find multiple subsequences at the same time.
I think this is REALLY good for primer design because it makes it easy to find restriction sites.

In [58]:
from Bio.Seq import Seq, MutableSeq
seq = Seq("GCCATTGTAATGGGCCGCTGAAAGGGTGCCCGA")

## Searching for the subsequence in the object seq.
##It returns the index of where it starts
##Will return -1 if the subsequence isn't found

seq.find("ATGGGCCGC")
print(seq.find("ATGGGCCGC"))
seq.find("ATGGCCCGC")
print(seq.find("ATGGCCCGC"))
seq.find("ATGGGCCGC")
## rfind differs from find by searching from right to left
print(seq.rfind("ATGGGCCGC"))
seq.find("ATGGCCCGC")
print(seq.rfind("ATGGCCCGC"))

for index, sub in seq.search(["ATG", "GGG", "CC"]):
    print(index, sub)

9
-1
9
-1
1 CC
9 ATG
11 GGG
14 CC
23 GGG
28 CC
29 CC


How to pull SeqRecord objects from a GenBank Files.

There is information about how to do this with FASTA files, but I don't like FASTA files. Mainly because they are sparse and do not have a standard format.



In [66]:

from Bio import SeqIO
record = SeqIO.read("NC_005816.gb.txt","genbank")
print(record)
len(record.annotations)

ID: NC_005816.1
Name: NC_005816
Description: Yersinia pestis biovar Microtus str. 91001 plasmid pPCP1, complete sequence
Database cross-references: Project:58037
Number of features: 41
/molecule_type=DNA
/topology=circular
/data_file_division=BCT
/date=21-JUL-2008
/accessions=['NC_005816']
/sequence_version=1
/gi=45478711
/keywords=['']
/source=Yersinia pestis biovar Microtus str. 91001
/organism=Yersinia pestis biovar Microtus str. 91001
/taxonomy=['Bacteria', 'Proteobacteria', 'Gammaproteobacteria', 'Enterobacteriales', 'Enterobacteriaceae', 'Yersinia']
/references=[Reference(title='Genetics of metabolic variations between Yersinia pestis biovars and the proposal of a new biovar, microtus', ...), Reference(title='Complete genome sequence of Yersinia pestis strain 91001, an isolate avirulent to humans', ...), Reference(title='Direct Submission', ...), Reference(title='Direct Submission', ...)]
/comment=PROVISIONAL REFSEQ: This record has not yet been subject to final
NCBI review. The 

13

2.4.1 Simple FASTA parsing Example.

I downloaded a .txt for this of the FASTA file about Lady Slipper Orchids as mentioned in the documentation, because when I returned the same search from NCBI now (6-26-2024) I got 81,992 results and I don't have that kind of RAM just lying around.

This may be silly:

To add the files I wanted I needed to upload them to the folder icon on the left panel of the screen.

In [13]:
from Bio import SeqIO
for seq_record in SeqIO.parse("ls_orchid.fasta.txt", "fasta"):
     print(seq_record.id)
     print(repr(seq_record.seq))
     print(len(seq_record))

gi|2765658|emb|Z78533.1|CIZ78533
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGG...CGC')
740
gi|2765657|emb|Z78532.1|CCZ78532
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAACAG...GGC')
753
gi|2765656|emb|Z78531.1|CFZ78531
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGCAG...TAA')
748
gi|2765655|emb|Z78530.1|CMZ78530
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAAACAACAT...CAT')
744
gi|2765654|emb|Z78529.1|CLZ78529
Seq('ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAA...AAA')
733
gi|2765652|emb|Z78527.1|CYZ78527
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...CCC')
718
gi|2765651|emb|Z78526.1|CGZ78526
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...TGT')
730
gi|2765650|emb|Z78525.1|CAZ78525
Seq('TGTTGAGATAGCAGAATATACATCGAGTGAATCCGGAGGACCTGTGGTTATTCG...GCA')
704
gi|2765649|emb|Z78524.1|CFZ78524
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGATAGTAG...AGC')
740
gi|2765648|emb|Z78523.1|CHZ78523
Seq('CGTAACCAGGTTTCCGT

In [32]:
from Bio import SeqIO
for seq_record in SeqIO.parse("ls_orchid.gbk.txt", "genbank"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))

Z78533.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGG...CGC')
740
Z78532.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAACAG...GGC')
753
Z78531.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGCAG...TAA')
748
Z78530.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAAACAACAT...CAT')
744
Z78529.1
Seq('ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAA...AAA')
733
Z78527.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...CCC')
718
Z78526.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAG...TGT')
730
Z78525.1
Seq('TGTTGAGATAGCAGAATATACATCGAGTGAATCCGGAGGACCTGTGGTTATTCG...GCA')
704
Z78524.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGATAGTAG...AGC')
740
Z78523.1
Seq('CGTAACCAGGTTTCCGTAGGTGAACCTGCGGCAGGATCATTGTTGAGACAGCAG...AAG')
709
Z78522.1
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGCAG...GAG')
700
Z78521.1
Seq('GTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAGTAGAATATATGATCGAGT...ACC')
726
Z78520.1
Seq('CGTAACAAGGTTTC